In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Read data

In [3]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

# Generate X Y 

In [4]:
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(604, 1704)
(604, 5112)


In [5]:
x1Restruct_Fun=x1
x2Restruct_Fun=x2
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(604, 1704)
(604, 5112)


In [6]:
YRestruct_Fun=np.zeros([601,5112])
for j in range (0,71):
    for i in range(0,601):
        YRestruct_Fun[i][72*j:72*j+24]=x1Restruct_Fun[1+i][24*j:24*j+24]
        YRestruct_Fun[i][72*j+24:72*j+48]=x1Restruct_Fun[1+i+1][24*j:24*j+24]
        YRestruct_Fun[i][72*j+48:72*j+72]=x1Restruct_Fun[1+i+2][24*j:24*j+24]
print(np.shape(YRestruct_Fun))

(601, 5112)


In [7]:
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(601, 6816)


In [8]:
Xhat=np.zeros([540,6816])
Yhat=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        Xhat[i][j]=XRestruct_Fun[i][j]
        Yhat[i][j]=YRestruct_Fun[i][j]
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(540, 6816)
Yhat
(540, 5112)


# Expend X Y

In [9]:
x1=np.zeros([540,1704])
x2=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,1704):
        x1[i][j]=Xhat[i][j]
    for j in range (0,5112):
        x2[i][j]=Xhat[i][1704+j]
print(np.shape(x1))
print(np.shape(x2))

(540, 1704)
(540, 5112)


In [10]:
new_x=np.zeros([540*72,6816]) #38880
new_y=np.zeros([540*72,5112]) #38880
#new_x : 0th~1703th column
for i in range (0,72):
    for j in range (0,540):
        for k in range (0,1704):
            new_x[540*i+j][k]=x1[j][k]
#new_x : 1704th~8519th column
for i in range (0,72):
    for j in range (0,540):
        for k in range (0+i,5040+1+i,72): #5040=5112-72
            new_x[540*i+j][1704+k]=x2[j][k]
#new_y
for i in range (0,72):
    for j in range (0,540):
        for k in range (0,5112):
            new_y[540*i+j][k]=Yhat[j][k]         
            
print(np.shape(new_x))
print(np.shape(new_y))

(38880, 6816)
(38880, 5112)


# PCA & Linear regression

In [11]:
per = np.zeros(99)
for i in range (0,99):
    per[i]=0.01*(i+1)
per_error1 = np.zeros(99)
per_error2 = np.zeros(99)

In [12]:
for z in range (0,99):
    pca = PCA(n_components=per[z])
    pca.fit(new_x)
    x_pca=pca.transform(new_x)
    model = LinearRegression()
    model.fit(x_pca, new_y)
    fit_y=model.predict(x_pca)
    
    """
    generate new cal
    """
    x1=np.zeros([604,1704])
    x2=np.zeros([604,5112])

    #x1 (0th~1703th column as x)
    for i in range (0,604):
        for j in range (0,71):
            a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
            for k in range (0,24):
                x1[i][j*24+k]=a[k]

    #x2 (1704th~6816th column as x)
    for i in range (1,604):
        b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        for j in range(0,5112):
            x2[i-1][j]=b[j]
    x1Restruct_Fun=x1
    x2Restruct_Fun=x2
    
    XRestruct_Fun=np.zeros([601,6816])
    for i in range (0,601):
        for j in range (0,1704):
            XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
        for j in range (1704,6816):
            XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]

    xf = np.zeros([61,6816])
    for i in range (0,61):
        for j in range(0,6816):
            xf[i][j] = XRestruct_Fun[i+540][j]

    x1 = np.zeros([61,1704])
    x2 = np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,1704):
            x1[i][j] = xf[i][j]
        for j in range (1704,6816):
            x2[i][j-1704] = xf[i][j]

    new_exx=np.zeros([61*72,6816]) 
    #new_x : 0th~1703th column
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0,1704):
                new_exx[61*i+j][k]=x1[j][k]
    #new_x : 1704th~8519th column
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0+i,5040+1+i,72):  #5040=5112-72
                new_exx[61*i+j][1704+k]=x2[j][k]

    new_xcal=pca.transform(new_exx)
    fit_y=model.predict(new_xcal)

    fit=np.zeros([61,5112])
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0,71):
                fit[j][72*k+i]=fit_y[61*i+j][72*k+i]

    fit_mean=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            for k in range (0,72):
                fit_mean[i][j]=fit_mean[i][j]+fit_y[61*k+i][j]/72

    fit_test=fit
    fit_mean_test=fit_mean

    """
    old data
    """
    old_x=np.zeros([601,5112])
    old_y=np.zeros([601,5112])
    #data_cal
    for i in range (1,602):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i-1]=a
    #data_obs
    for i in range (1,602):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i-1]=a

    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    fit_fx2mean=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+540][j]
            old_y2[i][j]=old_y[i+540][j]
            fit_fx2[i][j]=fit_test[i][j]
            fit_fx2mean[i][j]=fit_mean_test[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    after_mean2=abs(fit_fx2mean-old_y2)

    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)
    after_error_mean2=np.nanmean(after_mean2)

    per_error1[z] = after_error2
    per_error2[z] = after_error_mean2

# Result

In [13]:
print(per_error1)

[5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401
 5.95330401 5.95330401 5.95330401 5.95330401 5.95330401 5.88168356
 5.88168356 5.88168356 5.88168356 5.88168356 5.88168356 5.88168356
 5.88168356 5.88168356 5.88168356 5.88168356 5.83797158 5.83797158
 5.83797158 5.8589268  5.8589268  5.8589268  5.81953121 5.81953121
 5.73568638 5.75321046 5.74692524 5.7379413  5.72726733 5.68850845
 5.77421415 5.77190631 5.78565097 5.71669198 5.71070462 5.73625757
 5.75219546 5.77102279 5.78087523 5.81519245 5.8461396  5.9013

In [15]:
print('data compression with time on', np.argmin(per_error1)+1, '% PCA', 'have the min MAE error :', min(per_error1))
# print(0.01*(np.argmin(per_error1)+1))
# print(min(per_error1))

data compression with time on 78 % PCA have the min MAE error : 5.688508445648077


In [16]:
print(per_error2)

[5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248
 5.95334248 5.95334248 5.95334248 5.95334248 5.95334248 5.88173503
 5.88173503 5.88173503 5.88173503 5.88173503 5.88173503 5.88173503
 5.88173503 5.88173503 5.88173503 5.88173503 5.83802516 5.83802516
 5.83802516 5.85897091 5.85897091 5.85897091 5.81956461 5.81956461
 5.73578637 5.75334216 5.74705016 5.7380668  5.72741892 5.68873995
 5.77441668 5.7721433  5.78603256 5.71718823 5.71129316 5.73724017
 5.75230189 5.76322794 5.77194599 5.79295541 5.79739666 5.8549

In [17]:
print('data compression with mean on', np.argmin(per_error2)+1, '% PCA', 'have the min MAE error :', min(per_error2))
# print(0.01*(np.argmin(per_error2)+1))
# print(min(per_error2))

data compression with mean on 78 % PCA have the min MAE error : 5.68873995192551
